## k-means 군집분석 (비계층적 군집분석)

임의의 k개의 점을 기반으로 가까운 거리의 데이터를 묶는 것과 더불어 평균을 활용하는 군집분석 기법

군집 개수(k)를 확정하기 위해 여러 번의 시행착오 필요

결과 고정을 위해 seed(random_state) 설정 필요

#### sklearn - MinMaxScaler()
MinMax 정규화를 실시하는 sklearn 함수

fit() 메서드로 규칙 모델을 만들고 transform() 함수로 변환 실시

#### sklearn - StandardScaler()
표준화를 실시하는 sklearn 함수

fit() 메서드로 규칙 모델을 만들고 transform() 함수로 변환 실시

#### sklearn - KMeans()
k-means 군집분석을 실시하는 sklearn 함수

n_clusters, max_iter, random_state에 각각 군집 개수, 최대 반복 연산, 결과 고정 설정 가능

KMeans() 함수의 fit() 메서드에 데이터를 할당하여 학습 진행

결과 객체의 cluster_centers_와 labels_ 어트리뷰트로 군집 중심과 각 행의 군집 번호 확인 가능

In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [2]:
iris = pd.read_csv('ex/iris.csv')
df_1 = iris.head()
df_2 = iris.tail(1)

In [3]:
nor_minmax = MinMaxScaler().fit(df_1.iloc[:,:-1])
df_minmax = pd.DataFrame(nor_minmax.transform(df_1.iloc[:,:-1]),
                         columns = df_1.columns[:-1])

nor_minmax.transform(df_1.iloc[:,:-1])
nor_minmax.transform(df_2.iloc[:,:-1])

# row가 하나라면 normalization 작동 X
nor_minmax = MinMaxScaler().fit(df_2.iloc[:,:-1])
nor_minmax.transform(df_2.iloc[:,:-1])

array([[0., 0., 0., 0.]])

In [4]:
# k-means
kmeans = KMeans(n_clusters=3, random_state=123, n_init=10)
model = kmeans.fit(iris.iloc[:,:-1])

model.labels_ # label 결과
model.cluster_centers_ # 중심점 값

iris['cluster'] = model.labels_ # 클러스터링 결과 각 데이터가 몇 번째 그룹에 속하는지 저장
iris.groupby('cluster').mean(numeric_only=True).reset_index()

C:\Users\home\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


,cluster,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,0,6.850000,3.073684,5.742105,2.071053
1,1,5.006000,3.428000,1.462000,0.246000
2,2,5.901613,2.748387,4.393548,1.433871


In [5]:
# BMI가 0이 아닌 사람 데이터를 대상으로 k-means 군집 분석을 실시하는 경우 군집 개수가 가장 큰 군집의 Insulin 평균은 얼마인가?
diabetes = pd.read_csv('ex/diabetes.csv')
diabetes = diabetes[diabetes['BMI']!=0]

kmeans = KMeans(n_clusters=4, random_state=123, n_init=10)
model = kmeans.fit(diabetes)

diabetes['cluster'] = model.labels_
# diabetes_count = diabetes.groupby('cluster')['Insulin'].count().reset_index()
diabetes_count = diabetes['cluster'].value_counts().reset_index()
diabetes_mean = diabetes.groupby('cluster')['Insulin'].mean().reset_index()

clustered_diabetes = pd.merge(left=diabetes_count, right=diabetes_mean, left_on='cluster', right_on='cluster')
round(clustered_diabetes[clustered_diabetes['count'] == clustered_diabetes['count'].max()]['Insulin'], 2)

C:\Users\home\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


0    2.96
Name: Insulin, dtype: float64

In [6]:
# BMI가 0이 아닌 사람 데이터를 대상으로 k-means 군집 분석을 실시하는 경우 군집 개수가 가장 큰 군집의 나이 평균은 얼마인가?
diabetes = pd.read_csv('ex/diabetes.csv')
diabetes = diabetes[diabetes['BMI']!=0]

minmax_diabetes = MinMaxScaler().fit(diabetes)
nor_diabetes = minmax_diabetes.transform(diabetes)
nor_diabetes = pd.DataFrame(nor_diabetes,
                            columns=diabetes.columns)
nor_diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0.352941,0.743719,0.590164,0.353535,0.000000,0.314928,0.234415,0.483333,1.0
1,0.058824,0.427136,0.540984,0.292929,0.000000,0.171779,0.116567,0.166667,0.0
2,0.470588,0.919598,0.524590,0.000000,0.000000,0.104294,0.253629,0.183333,1.0
3,0.058824,0.447236,0.540984,0.232323,0.111111,0.202454,0.038002,0.000000,0.0
4,0.000000,0.688442,0.327869,0.353535,0.198582,0.509202,0.943638,0.200000,1.0


In [7]:
nor_model = KMeans(n_clusters=4, random_state=123, n_init=10).fit(nor_diabetes)
diabetes['cluster'] = nor_model.labels_
diabetes.head()

C:\Users\home\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,cluster
0,6,148,72,35,0,33.6,0.627,50,1,0
1,1,85,66,29,0,26.6,0.351,31,0,1
2,8,183,64,0,0,23.3,0.672,32,1,0
3,1,89,66,23,94,28.1,0.167,21,0,1
4,0,137,40,35,168,43.1,2.288,33,1,2


In [8]:
cluster_count = diabetes['cluster'].value_counts().reset_index()
cluster_age = diabetes.groupby('cluster')['Age'].mean().reset_index()

clustered_diabetes = pd.merge(left=cluster_count, right=cluster_age, left_on='cluster', right_on='cluster')
clustered_diabetes
# clustered_diabetes[clustered_diabetes['count'] == clustered_diabetes['count'].max()]

,cluster,count,Age
0,1,360,25.630556
1,2,135,29.755556
2,0,131,44.526718
3,3,131,46.694656


In [13]:
# BMI가 0이 아닌 사람 데이터를 대상으로 k-means 군집 분석을 실시하고 군집의 중심점간 유클리드 거리가 가장 가까운 그룹간 거리는?
diabetes = pd.read_csv('ex/diabetes.csv')
diabetes = diabetes[diabetes['BMI']!=0]

model = KMeans(n_clusters=3, random_state=123, n_init=10).fit(diabetes)
df_centers = pd.DataFrame(model.cluster_centers_, columns=diabetes.columns)
df_centers = df_centers.transpose()
df_centers

C:\Users\home\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


,0,1,2
Pregnancies,3.975258,4.026316,3.542735
Glucose,114.237113,158.447368,129.376068
BloodPressure,68.647423,72.000000,71.478632
SkinThickness,15.259794,32.263158,30.337607
Insulin,14.696907,441.289474,159.401709
BMI,31.440619,35.107895,34.134615
DiabetesPedigreeFunction,0.434579,0.569211,0.535188
Age,33.808247,34.763158,31.948718
Outcome,0.301031,0.578947,0.418803


In [14]:
# 유클리드 거리
print(sum((df_centers.iloc[:,0] - df_centers.iloc[:,1]) ** 2) ** 0.5)
print(sum((df_centers.iloc[:,1] - df_centers.iloc[:,2]) ** 2) ** 0.5)
print(sum((df_centers.iloc[:,2] - df_centers.iloc[:,0]) ** 2) ** 0.5)

429.24419310888464
283.405999774738
146.33847909815492
